## Perguntas

1) Correlação das variaveis
2) Qual estilo de jogo é mais provavel de se ter melhores vendas?
3) Qual plataforma é provavel de se ter melhores vendas?
4)  Aplicar um modelo de regressão ou classificação utilizando uma técnica de hiperparametrização automática;</br>
5)  Aplicar uma medida de avaliação do modelo.
6)  Agrupamento: gerar a visualização Scatter
7)  Classificação: gerar a matriz de confusão
8)  Regressão: gerar a visualização da linha de saída

In [73]:
#IMPORT E INSTALL AS BIBLIOTECAS IMPORTANTES
!pip install googletrans==4.0.0-rc1

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from googletrans import Translator

In [74]:
caminho = 'Data/vgchartz-2024.csv'
jogos = pd.read_csv(caminho)

In [86]:
jogos_tratados = jogos.drop(columns=['img'])
jogos_tratados.head(3)

,title,console,genre,publisher,developer,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
0,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,9.4,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
1,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,9.7,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03
2,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,9.6,16.15,8.41,0.47,5.49,1.78,2002-10-28,NaN


In [87]:
#TRADUZIR OS NOMES EM INGLES PARA PT-BR

def traduzir_colunas(df, language='pt'):
    translator = Translator()
    traducoes = {}
    falha_ao_traduzir = {
        'publisher': 'Editora',
        'developer': 'Desenvolvedora',
        'na_sales': 'Vendas_na',
        'jp_sales': 'Vendas_jp',
        'pal_sales': 'Vendas_pal',
        'other_sales': 'Vendas_demais_regiões',
        'critic_score': 'Pontuação_Geral'
    }
    # Traduzir todas as colunas
    for column in df.columns:
        try:
            if column.lower() in falha_ao_traduzir:
                traducoes[column] = falha_ao_traduzir[column.lower()].capitalize()  # Utilize a tradução personalizada
            else:
                translated = translator.translate(column, src='en', dest=language)
                capslock = translated.text.capitalize()  # Maiuscula na primeira letra de cada palavra
                traducoes[column] = capslock
        except Exception as e:
            print(f"Erro ao traduzir a coluna '{column}': {e}")
            traducoes[column] = column  # Manter o nome original em caso de erro
    df.columns = [traducoes.get(col, col) for col in df.columns]

In [88]:
#EXECUTA A FUNÇÃO DE TRADUÇÃO
traduzir_colunas(jogos_tratados)
jogos_tratados.head(3)

,Título,Console,Gênero,Editora,Desenvolvedora,Pontuação_geral,Vendas totais,Vendas_na,Vendas_jp,Vendas_pal,Vendas_demais_regiões,Data de lançamento,Última atualização
0,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,9.4,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
1,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,9.7,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03
2,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,9.6,16.15,8.41,0.47,5.49,1.78,2002-10-28,NaN


In [89]:
jogos_tratados.isna().sum()

Título                       0
Console                      0
Gênero                       0
Editora                      0
Desenvolvedora              17
Pontuação_geral          57338
Vendas totais            45094
Vendas_na                51379
Vendas_jp                57290
Vendas_pal               51192
Vendas_demais_regiões    48888
Data de lançamento        7051
Última atualização       46137
dtype: int64

In [93]:
valores_ausentes = jogos_tratados[jogos_tratados['Desenvolvedora'].isna()]
valores_ausentes.head(18)

,Título,Console,Gênero,Editora,Desenvolvedora,Pontuação_geral,Vendas totais,Vendas_na,Vendas_jp,Vendas_pal,Vendas_demais_regiões,Data de lançamento,Última atualização
8486,Gourmet Chef: Cook Your Way to Fame,DS,Misc,Ubisoft,NaN,NaN,0.14,0.13,NaN,NaN,0.01,2008-06-24,NaN
10832,Wordmaster,DS,Strategy,"Destination Software, Inc",NaN,NaN,0.09,0.09,NaN,0.0,0.01,2008-08-12,NaN
18185,SAS: Secure Tomorrow,PC,Shooter,City Interactive,NaN,NaN,0.00,NaN,NaN,0.0,0.00,2008-10-16,NaN
18455,My Baby and Me,Wii,Misc,Unknown,NaN,NaN,0.00,NaN,NaN,0.0,NaN,NaN,NaN
19925,Bounty Hunter,PC,Action,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999-11-30,NaN
25045,Abandoner,PC,Adventure,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004-09-17,NaN
27407,The Enchanted Unicorn,PC,Adventure,Take-Two Interactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-09-16,NaN
31230,istanbul Beyleri,PC,Fighting,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-12-17,NaN
37057,Purrfect Pet Shop,PC,Misc,eGames,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-06-13,NaN
40753,Duludubi Star,PC,Platform,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
